In [9]:
import pandas as pd
import numpy as np

#import plotly.express as px
import folium
from folium import Map
from folium.plugins import HeatMap
from IPython.display import Image

from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('/home/starczyn/Solar_Project/data/TTS_data.csv')

/tmp/ipykernel_2327/3678986517.py:1: DtypeWarning: Columns (2,3,13,28,29,30,31,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/home/starczyn/Solar_Project/data/TTS_data.csv')


In [3]:
data.keys()

Index(['Unnamed: 0', 'dataProvider1', 'dataProvider2', 'program1ProjectID',
       'PTODate_orProxy_', 'systemSizeInDCSTC_KW_', 'totalInstalledCost___',
       'Up_FrontCashIncentive___', 'customerSegment', 'is_expansion',
       'is_multiphase', 'tracking_Y_N_', 'ground_mounted_Y_N_',
       'hostCustomerZip__4_', 'hostCustomerCity', 'state',
       'city_state_country', 'latitude', 'longitude',
       'utilityServiceTerritory', 'x3rdPartyOwned_1_0_', 'installer_clean',
       'self_install_Y_N_', 'azimuth_1', 'tilt_1', 'mod_manuf1_clean',
       'mod_model1_clean', 'moduleQty_1', 'mod_manuf2_clean',
       'mod_model2_clean', 'mod_manuf3_clean', 'mod_model3_clean',
       'additionalModuleModels_Y_N_', 'mod_tech1', 'mod_tech2', 'mod_tech3',
       'mod_BIPV1', 'mod_bifacial1', 'mod_nameplate_capacity1',
       'mod_efficiency1', 'inv_manuf1_clean', 'inv_model1_clean',
       'inverterQuantity_1', 'additionalInverterModels_Y_N_', 'inv_microinv1',
       'inv_battery_hybrid1', 'inv_bui

In [4]:
data = data.dropna()
#data.describe()

In [13]:
data['eff_scaled'] = data['mod_efficiency1']/data['mod_efficiency1'].abs().max()
data['eff_scaled'].mean()


0.7872994107793959

Location Heat Map

In [ ]:
map = Map(
    location=[39.8283, -98.5795],
    tiles="cartodbpositron",
    zoom_start= 4.25,
)

data_samp = data.sample(frac = .0005)
data_samp = data_samp.dropna()
len(data_samp)

In [ ]:
hm_wide = HeatMap(
    list(zip(data.latitude.values, data.longitude.values)),
    min_opacity=0.5,
    radius=4, 
    blur=5, 
    max_zoom=10,
)

In [ ]:
map.add_child(hm_wide)

Chloropleths

Average efficiency per state

In [102]:


url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_geo = f"{url}/us-states.json"

map = Map(
    location=[39.8283, -98.5795],
    #tiles="cartodbpositron",
    zoom_start= 4.25,
)

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=data,
    columns=["state", "mod_efficiency1"],
    key_on="feature.id",
    fill_color="BuPu",
    fill_opacity=0.7,
    line_opacity=0.2,
    bins = 9,
    legend_name="Efficiency (%)",
).add_to(map)

folium.LayerControl().add_to(map)

map

In [112]:
def get_eff_average(data):
    state_array = data.state.unique()
    average_eff = []
    for state in state_array:
        state_df = data[data.state == state]
        avg_eff = state_df['mod_efficiency1'].mean()
        state_df = data[data.state == state]
        average_eff.append(avg_eff)

    return(average_eff)

In [113]:
len(get_eff_average(data))
data.state.unique()

array(['AZ', 'CT', 'TX', 'WA', 'NY', 'OR', 'FL', 'DC', 'MA', 'NH', 'RI',
       'AR', 'CA', 'MN', 'NM', 'PA', 'VT', 'UT', 'WI'], dtype=object)

In [114]:
states = data.state.unique()
avg_eff = get_eff_average(data)
avg_eff_df = pd.DataFrame({'state': states,'avg_eff': avg_eff})
avg_eff_df

,state,avg_eff
0,AZ,0.172704
1,CT,0.179696
2,TX,0.175628
3,WA,0.182126
4,NY,0.180233
5,OR,0.178636
6,FL,0.152507
7,DC,0.179381
8,MA,0.174540
9,NH,0.170612


In [122]:
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_geo = f"{url}/us-states.json"

map1 = Map(
    location=[39.8283, -98.5795],
    #tiles="cartodbpositron",
    zoom_start= 4.25,
)

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=avg_eff_df,
    columns=["state", "avg_eff"],
    key_on="feature.id",
    fill_color="BuPu",
    fill_opacity=0.7,
    line_opacity=0.2,
    bins = 8,
    legend_name="Efficiency (%)",
).add_to(map1)

folium.LayerControl().add_to(map1)

map1

Average Cost per State

In [46]:
cost_data = data[data.totalInstalledCost___ != -1]

In [63]:
cost_data.state.unique()

array(['AZ', 'CT', 'TX', 'WA', 'NY', 'OR', 'FL', 'MA', 'NH', 'RI', 'AR',
       'CA', 'MN', 'NM', 'PA', 'VT', 'UT', 'WI'], dtype=object)

In [69]:
def get_cost_average(data):
    state_array = cost_data.state.unique()
    average_costs = []
    for state in state_array:
        state_df = data[data.state == state]
        avg_cost = state_df['totalInstalledCost___'].mean()
        average_costs.append(avg_cost)

    return(average_costs)


In [88]:
states = cost_data.state.unique()
avg_costs = get_cost_average(cost_data)
avg_cost_df = pd.DataFrame({'state': states,'avg_cost': avg_costs})
avg_cost_df

,state,avg_cost
0,AZ,40820.341940
1,CT,31484.774078
2,TX,35740.410250
3,WA,39502.272948
4,NY,79645.370861
5,OR,22801.013562
6,FL,85082.843654
7,MA,76368.895928
8,NH,38820.864775
9,RI,71630.927571


Average Installation Cost per Module

In [127]:

url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_geo = f"{url}/us-states.json"

map2 = Map(
    location=[39.8283, -98.5795],
    #tiles="cartodbpositron",
    zoom_start= 4.25,
)

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=avg_cost_df,
    columns=["state", "avg_cost"],
    key_on="feature.id",
    fill_color="BuPu",
    fill_opacity=0.7,
    line_opacity=0.2,
    bins = 9,
    legend_name="Average Installation Cost per System, $",
).add_to(map2)

folium.LayerControl().add_to(map2)

map2

Total Cost of all installed systems:

In [91]:
def get_cost_total(data):
    state_array = cost_data.state.unique()
    total_costs = []
    for state in state_array:
        state_df = data[data.state == state]
        total_cost = state_df['totalInstalledCost___'].sum()
        total_costs.append(total_cost/1000000)

    return(total_costs)

In [92]:
states = cost_data.state.unique()
total_costs = get_cost_total(cost_data)
total_cost_df = pd.DataFrame({'state': states,'total_cost': total_costs})
total_cost_df

,state,total_cost
0,AZ,1284.656981
1,CT,1435.642728
2,TX,853.945622
3,WA,232.628885
4,NY,7921.210004
5,OR,36.937642
6,FL,165.315965
7,MA,7960.617343
8,NH,157.418607
9,RI,180.151783


In [120]:
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_geo = f"{url}/us-states.json"

map3 = Map(
    location=[39.8283, -98.5795],
    zoom_start= 4.25,
)

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=total_cost_df,
    columns=["state", "total_cost"],
    key_on="feature.id",
    fill_color="BuPu",
    fill_opacity=0.7,
    line_opacity=0.2,
    bins = 9,
    legend_name="Total Costs of all Systems, $ (millions)",
).add_to(map3)

folium.LayerControl().add_to(map3)

map3

In [138]:
import branca.colormap
from collections import defaultdict
import folium
import webbrowser
from folium.plugins import HeatMap 

map_osm = folium.Map(llocation=[35,110],zoom_start=1)

steps=20
colormap = branca.colormap.linear.YlOrRd_09.scale(0, 1).to_step(steps)
gradient_map=defaultdict(dict)
for i in range(steps):
    gradient_map[1/steps*i] = colormap.rgb_hex_str(1/steps*i)
colormap.add_to(map_osm) #add color bar at the top of the map

HeatMap(list(zip(data.latitude.values, data.longitude.values)), gradient = gradient_map).add_to(map_osm) # Add heat map to the previously created map

file_path = r"C:\\test.html"
map_osm.save(file_path) # Save as html file
webbrowser.open(file_path) # Default browser open

Start : This command cannot be run due to the error: The system cannot find the file specified.
At line:1 char:1
+ Start "C:\\test.html"
+ ~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [Start-Process], InvalidOperationException
    + FullyQualifiedErrorId : InvalidOperationException,Microsoft.PowerShell.Commands.StartProcessCommand
 


True